<a href="https://colab.research.google.com/github/LeoPiresDeSouza/DSAcademy-IAGenerativaProcessamentoLinguagemnatural/blob/master/Projeto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>IA Generativa e LLMs Para Processamento de Linguagem Natural</font>
## <font color='blue'>Projeto 3</font>
## <font color='blue'>Fine-Tuning de LLM Open-Source Para App de Assistente Jurídico</font>

## Instalando e Carregando Pacotes

In [8]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.4 MB/s eta 0:00:00


In [1]:
# Instalando os pacotes
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 51.9 MB/s eta 0:00:00


In [5]:
# Imports
import numpy as np
import nltk
import shutil
import evaluate
import transformers
import datasets
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [7]:
# Remove as pastas, se já existirem. Isso evita erros ao executar o jupyter a partir da segunda vez em diante
try:
    shutil.rmtree('logs_treino')
    shutil.rmtree('resultados_treino')
    shutil.rmtree('modelo_salvo')
except:
    print('As pastas não existem ou já foram removidas!')

As pastas não existem ou já foram removidas!


## Carregando os Dados

In [12]:
# Define o nome do arquivo
nome_arquivo = "dataset.csv"

load-dataset é uma biblioteca do Hugging Face

In [13]:
# Carrega os dados
dsa_dataset = load_dataset('csv', data_files = nome_arquivo, delimiter = ',')

FileNotFoundError: Unable to find '/content/dataset.csv'

O pacote permite dividir o arquivo de entrada em duas massas diferentes. Uma será utilizada para treino e outr será utilizada para testear o modelo treinado.

In [10]:
# Divisão em treino e teste com proporção 80/20
dsa_dataset = dsa_dataset["train"].train_test_split(test_size = 0.2)

NameError: name 'dsa_dataset' is not defined

In [ ]:
# Visualiza o formato do dataset
dsa_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 2993
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 749
    })
})

## Tokenizador e LLM Open-Source

https://huggingface.co/google/flan-t5-base

legacy = False é encessário para as novas versões do tokenizador para que ele trabalhe de acordo com os novos procedimentos.

Caso na documentação do modelo não cite o tokenizador, é necessário buscar na documentação do tokenizador.

In [ ]:
# Carrega o tokenizador
tokenizador = T5Tokenizer.from_pretrained("google/flan-t5-base", legacy = False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Visualiza o tokenizador
tokenizador

T5Tokenizer(name_or_path='google/flan-t5-base', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '

In [ ]:
# Carrega o LLM pré-treinado
modelo = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

In [ ]:
# Visualiza a arquitetura do LLM
modelo

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
# Data collator para concatenar tokenizador e modelo
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizador, model = modelo)

In [ ]:
# Visualiza o data collator
data_collator

DataCollatorForSeq2Seq(tokenizer=T5Tokenizer(name_or_path='google/flan-t5-base', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<

## Pré-Processamento

In [ ]:
# Todos os inputs vão receber como prefixo: "answer the question" (responda a questão)
prefix = "answer the question: "

In [ ]:
# Função de pré-processamento
def dsa_fn_preprocess(data):

    # Concatena o prefixo a cada pergunta na lista de perguntas fornecida em data["question"]
    inputs = [prefix + doc for doc in data["question"]]

    # Usa o tokenizer para converter as perguntas processadas em tokens com um comprimento máximo de 128,
    # truncando as que forem mais longas
    model_inputs = tokenizador(inputs, max_length = 128, truncation = True)

    # Tokeniza as respostas fornecidas em data["answer"] com um comprimento máximo de 512,
    # truncando as que forem mais longas
    labels = tokenizador(text_target = data["answer"], max_length = 512, truncation = True)

    # Adiciona os tokens das respostas como rótulos no dicionário de entradas do modelo
    model_inputs["labels"] = labels["input_ids"]

    # Retorna o dicionário processado contendo tanto as entradas tokenizadas quanto os rótulos
    return model_inputs

In [ ]:
# Aplica a função de pré-processamento ao dataset gerando o dataset tokenizado
dsa_dataset_tokenized = dsa_dataset.map(dsa_fn_preprocess, batched = True)

Map:   0%|          | 0/2993 [00:00<?, ? examples/s]

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

In [ ]:
# Visualiza o dataset tokenizado
dsa_dataset_tokenized

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2993
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 749
    })
})

In [ ]:
dsa_dataset_tokenized['train']['question'][0]

'Q: Involuntary Lien. Hi. I received a notice of an involuntary lien placed on my property. According to the paperwork, it is for a small claims court that I never attended. I never received a court date. The forms say don not contact the court but to contact the person who took me to small claims court. Question is; can I open a court for this to invetsigate? I dont want to contact the person named on the forms. It might be a scam? '

In [ ]:
dsa_dataset_tokenized['train']['answer'][0]

"A:In California, if you have received a notice of an involuntary lien for a small claims court judgment that you were unaware of, it is essential to verify its legitimacy. Start by checking the court records to confirm if a case was actually filed and a judgment entered against you. If the court has no record of such a case, the notice might be fraudulent. If a case exists and you were not properly served with a court date, you may have grounds to contest the judgment. You can file a motion to vacate the judgment in the small claims court where the case was heard, explaining your lack of knowledge about the court proceedings. It's important not to ignore the lien, as it can affect your property rights. Contacting an attorney for guidance is advisable to navigate the process effectively. They can help determine the validity of the lien and the best course of action."

In [ ]:
dsa_dataset_tokenized['train']['input_ids'][0]

[1525,
 8,
 822,
 10,
 1593,
 10,
 86,
 4571,
 14016,
 651,
 1414,
 35,
 5,
 2018,
 5,
 27,
 1204,
 3,
 9,
 2103,
 13,
 46,
 16,
 4571,
 14016,
 651,
 12452,
 2681,
 30,
 82,
 785,
 5,
 2150,
 12,
 8,
 16879,
 6,
 34,
 19,
 21,
 3,
 9,
 422,
 3213,
 1614,
 24,
 27,
 470,
 5526,
 5,
 27,
 470,
 1204,
 3,
 9,
 1614,
 833,
 5,
 37,
 2807,
 497,
 278,
 59,
 574,
 8,
 1614,
 68,
 12,
 574,
 8,
 568,
 113,
 808,
 140,
 12,
 422,
 3213,
 1614,
 5,
 11860,
 19,
 117,
 54,
 27,
 539,
 3,
 9,
 1614,
 21,
 48,
 12,
 16,
 162,
 17,
 7,
 23,
 5339,
 58,
 27,
 2483,
 241,
 12,
 574,
 8,
 568,
 2650,
 30,
 8,
 2807,
 5,
 94,
 429,
 36,
 3,
 9,
 13236,
 58,
 1]

In [ ]:
dsa_dataset_tokenized['train']['labels'][0]

[71,
 10,
 1570,
 1826,
 6,
 3,
 99,
 25,
 43,
 1204,
 3,
 9,
 2103,
 13,
 46,
 16,
 4571,
 14016,
 651,
 12452,
 21,
 3,
 9,
 422,
 3213,
 1614,
 7661,
 24,
 25,
 130,
 24111,
 13,
 6,
 34,
 19,
 1832,
 12,
 10446,
 165,
 31841,
 5,
 3273,
 57,
 6450,
 8,
 1614,
 3187,
 12,
 3606,
 3,
 99,
 3,
 9,
 495,
 47,
 700,
 5132,
 11,
 3,
 9,
 7661,
 5136,
 581,
 25,
 5,
 156,
 8,
 1614,
 65,
 150,
 1368,
 13,
 224,
 3,
 9,
 495,
 6,
 8,
 2103,
 429,
 36,
 24283,
 5,
 156,
 3,
 9,
 495,
 8085,
 11,
 25,
 130,
 59,
 3085,
 2098,
 28,
 3,
 9,
 1614,
 833,
 6,
 25,
 164,
 43,
 9808,
 12,
 4233,
 8,
 7661,
 5,
 148,
 54,
 1042,
 3,
 9,
 4644,
 12,
 409,
 8367,
 8,
 7661,
 16,
 8,
 422,
 3213,
 1614,
 213,
 8,
 495,
 47,
 1943,
 6,
 13834,
 39,
 2136,
 13,
 1103,
 81,
 8,
 1614,
 13339,
 5,
 94,
 31,
 7,
 359,
 59,
 12,
 9751,
 8,
 12452,
 6,
 38,
 34,
 54,
 2603,
 39,
 785,
 2166,
 5,
 3235,
 53,
 46,
 4917,
 21,
 4864,
 19,
 3,
 22468,
 12,
 7939,
 8,
 433,
 3762,
 5,
 328,
 54,
 199,
 2082,
 8,


## Definindo a Métrica de Avaliação do Modelo

In [ ]:
# O pacote "punkt" é específico para a tarefa de tokenização, que envolve a divisão de um texto
# em uma lista de sentenças
nltk.download("punkt", quiet = True)

True

In [ ]:
# Define a métrica
metric = evaluate.load("rouge")

In [ ]:
# Função de cálculo das métricas
def dsa_calcula_metricas(eval_preds):

    # Desempacota as predições e os rótulos do argumento eval_preds
    previsoes, labels = eval_preds

    # Substitui todos os valores diferentes de -100 em labels pelo ID do token de preenchimento
    labels = np.where(labels != -100,
                      labels,
                      tokenizador.pad_token_id)

    # Decodifica as previsões para texto, ignorando tokens especiais
    previsoes_decodificadas = tokenizador.batch_decode(previsoes,
                                                       skip_special_tokens = True)

    # Decodifica os labels para texto, ignorando tokens especiais
    labels_decodificados = tokenizador.batch_decode(labels,
                                                    skip_special_tokens = True)

    # Adiciona uma nova linha após cada sentença para as previsões decodificadas,
    # preparando-as para a avaliação ROUGE
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in previsoes_decodificadas]

    # Adiciona uma nova linha após cada label para as previsões decodificadas,
    # preparando-os para a avaliação ROUGE
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in labels_decodificados]

    # Calcula a métrica ROUGE entre as previções e os rótulos decodificados, utilizando um stemmer
    resultado = metric.compute(predictions = previsoes_decodificadas,
                               references = labels_decodificados,
                               use_stemmer = True)

    # Retorna o resultado da métrica ROUGE
    return resultado

In [ ]:
# Define argumentos de treino
dsa_training_args = Seq2SeqTrainingArguments(output_dir = "resultados_treino",
                                             evaluation_strategy = "epoch",
                                             learning_rate = 3e-4,
                                             logging_dir = "logs_treino",
                                             logging_steps = 1,
                                             per_device_train_batch_size = 4,
                                             per_device_eval_batch_size = 2,
                                             weight_decay = 0.01,
                                             save_total_limit = 3,
                                             num_train_epochs = 3,
                                             predict_with_generate = True,
                                             push_to_hub = False)

In [ ]:
# Define o trainer
dsa_trainer = Seq2SeqTrainer(model = modelo,
                             args = dsa_training_args,
                             train_dataset = dsa_dataset_tokenized["train"],
                             eval_dataset = dsa_dataset_tokenized["test"],
                             tokenizer = tokenizador,
                             data_collator = data_collator,
                             compute_metrics = dsa_calcula_metricas)

## Treinamento do Modelo

In [ ]:
%%time
dsa_trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.741900,2.452794,0.127673,0.029235,0.102774,0.102892
2,2.252300,2.422110,0.129462,0.031206,0.105452,0.105480
3,2.713900,2.415890,0.129887,0.030162,0.104886,0.104961


CPU times: user 15min 53s, sys: 56.4 s, total: 16min 49s
Wall time: 17min 2s


TrainOutput(global_step=2247, training_loss=2.4560269389515406, metrics={'train_runtime': 1022.2189, 'train_samples_per_second': 8.784, 'train_steps_per_second': 2.198, 'total_flos': 1467591484050432.0, 'train_loss': 2.4560269389515406, 'epoch': 3.0})

In [ ]:
# Salva o modelo
dsa_trainer.save_model('modelo_salvo')

**ROUGE-1** mede a sobreposição de unigramas (palavras individuais).

**ROUGE-2** mede a sobreposição de bigramas (pares de palavras consecutivas).

**ROUGE-L** mede a sobreposição da subsequência mais longa comum entre o resumo gerado e o resumo de referência. Isso leva em conta a ordem das palavras, mas permite lacunas. Valores mais altos indicam melhor desempenho. ROUGE-L é calculado com base na similaridade entre as sequências, levando em consideração a precisão, o recall e a média harmônica entre eles.

Valores mais altos de ROUGE indicam uma maior semelhança entre o resumo gerado e o resumo de referência, o que geralmente é interpretado como uma indicação de melhor qualidade do resumo. No entanto, é importante lembrar que nenhuma métrica pode capturar completamente a qualidade de um resumo e é útil complementar a avaliação com análises qualitativas ou outras métricas.

## Deploy e Uso do Modelo

In [ ]:
# Carrega o tokenizador final salvo em disco
tokenizador_dsa_final = AutoTokenizer.from_pretrained('modelo_salvo')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Carrega o modelo final salvo em disco
modelo_dsa_final = AutoModelForSeq2SeqLM.from_pretrained('modelo_salvo')

In [ ]:
# Visualiza a arquitetura do modelo
modelo_dsa_final

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
# Texto de entrada
texto_entrada = "Can I move out of state with my children if I have a custody agreement in that state?"

In [ ]:
# Tokeniza o input
texto_entrada_tokenizado = tokenizador_dsa_final(texto_entrada, return_tensors = "pt").input_ids

In [ ]:
# Gera a saída (previsão do modelo)
texto_saida_tokenizado = modelo_dsa_final.generate(texto_entrada_tokenizado,
                                                   max_length = 50,
                                                   temperature = 0.4,
                                                   do_sample = True)

In [ ]:
# Visualiza
texto_saida_tokenizado

tensor([[    0,    71, 16701,  2791,    19,     3,     9, 13595, 11293,  2791,
           344,   192,  1362,     5,   156,    25,    43,    46,  2791,    16,
           286,    24,  1250,    25,    12,   888,    91,    13,   538,    28,
            39,   502,     6,    25,   164,    36,     3,   179,    12,   888,
            91,    13,   538,    28,    39,   502,     5,   611,     6,     3]])

In [ ]:
# Decode da saída
texto_saida = tokenizador_dsa_final.decode(texto_saida_tokenizado[0],
                                           skip_special_tokens = True)

In [ ]:
# Resultado
print("Pergunta:", texto_entrada)
print("Resposta:", texto_saida)

Pergunta: Can I move out of state with my children if I have a custody agreement in that state?
Resposta: A custody agreement is a legally binding agreement between two parents. If you have an agreement in place that allows you to move out of state with your children, you may be able to move out of state with your children. However, 


In [ ]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [ ]:
#%watermark -v -m

In [ ]:
#%watermark --iversions

# Fim